In [ ]:
from datasets import load_dataset
from transformers import AutoProcessor

In [2]:
model_name = "google/gemma-3-12b-it"
processor = AutoProcessor.from_pretrained(model_name, use_fast=True)
tokenizer = processor.tokenizer

In [3]:
dataset = load_dataset("HuggingFaceH4/Bespoke-Stratos-17k", split="train")

In [4]:
def format_restruction(dataset):
    system = dataset["system"]
    question = dataset["conversations"][0]["value"]
    response = dataset["conversations"][1]["value"]

    return {"system": system, "question": question, "response": response}

In [5]:
restructured_dataset = dataset.map(
    format_restruction,
    remove_columns=dataset.column_names,  # 기존 컬럼 제거
)

In [10]:
def filter_long_tokens(dataset):
    return (
        len(tokenizer.tokenize(dataset["system"]))
        + len(tokenizer.tokenize(dataset["question"]))
        + len(tokenizer.tokenize(dataset["response"]))
        <= 2044
    )

In [11]:
filtered_dataset = restructured_dataset.filter(
    filter_long_tokens, num_proc=6, desc="Filtering dataset"
)

Filtering dataset (num_proc=4):   0%|          | 0/16610 [00:00<?, ? examples/s]

In [13]:
filtered_dataset.push_to_hub(
    "Yeongi/Bespoke-Stratos-3.65k",
    private=False,  # 공개 여부
    # token="hf..",
    commit_message="Upload filtered dataset with max_seq_length <= 2044",
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Yeongi/Bespoke-Stratos-3.65k/commit/e2cd1425c93c5880f9031dd8b0467888c0aca056', commit_message='Upload filtered dataset with max_seq_length <= 2044', commit_description='', oid='e2cd1425c93c5880f9031dd8b0467888c0aca056', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Yeongi/Bespoke-Stratos-3.65k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Yeongi/Bespoke-Stratos-3.65k'), pr_revision=None, pr_num=None)